In [26]:
import consts, test_helper
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn, scipy, requests

In [27]:
importlib.reload(consts)
importlib.reload(test_helper)

<module 'test_helper' from '/Users/mike/Documents/School/Fa23/Clinic/code/test_helper.py'>

In [28]:
# USER
USER = "RYAN"
FILE_PATH = consts.PATH_MAP[USER]

ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

TRAIN_START_DATE = "20150101"
TRAIN_END_DATE = "20150601" # Up to but not including
TEST_START_DATE = "20150701"
TEST_END_DATE = "20150801"

# REGRESSION_TYPES = helper.Regression('OLS').list_all_regression_types()
# REGRESSION_TYPES

In [29]:
list_of_interacting_terms = [["relvol_nt_0","rrirpnxm_nt_0"], 
                            ["relvol_lst15_0", "rrirpnxm_lst15_0"],
                            ["relvol_lsthrx15_0", "rrirpnxm_lsthrx15_0"],
                            ["relvol_toxhr_0", "rrirpnxm_toxhr_0"]]

FEATURE_COL_NAMES = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0", "rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0"]

In [30]:
importlib.reload(test_helper)
data = test_helper.Data(FILE_PATH)
train_df = data.update_and_get_train_df(FILE_PATH, "20190701", movingBack_dayCount = 31, years_count = 1)

No YYYYMMDD datetime matched.

Filtered File Dates: ['data.20180531_1200', 'data.20180601_1200', 'data.20180604_1200', 'data.20180605_1200', 'data.20180606_1200', 'data.20180607_1200', 'data.20180608_1200', 'data.20180611_1200', 'data.20180612_1200', 'data.20180613_1200', 'data.20180614_1200', 'data.20180615_1200', 'data.20180618_1200', 'data.20180619_1200', 'data.20180620_1200', 'data.20180621_1200', 'data.20180622_1200', 'data.20180625_1200', 'data.20180626_1200', 'data.20180627_1200', 'data.20180628_1200', 'data.20180629_1200', 'data.20180702_1200', 'data.20180703_1200', 'data.20180705_1200', 'data.20180706_1200', 'data.20180709_1200', 'data.20180710_1200', 'data.20180711_1200', 'data.20180712_1200', 'data.20180713_1200', 'data.20180716_1200', 'data.20180717_1200', 'data.20180718_1200', 'data.20180719_1200', 'data.20180720_1200', 'data.20180723_1200', 'data.20180724_1200', 'data.20180725_1200', 'data.20180726_1200', 'data.20180727_1200', 'data.20180730_1200', 'data.20180731_1200', '

In [31]:
importlib.reload(test_helper)
test_dfs = data.get_df_between_date(data_path = FILE_PATH, start_date="20150701", end_date="20150801")

Filtered File Dates: ['data.20150701_1200', 'data.20150702_1200', 'data.20150706_1200', 'data.20150707_1200', 'data.20150708_1200', 'data.20150709_1200', 'data.20150710_1200', 'data.20150713_1200', 'data.20150714_1200', 'data.20150715_1200', 'data.20150716_1200', 'data.20150717_1200', 'data.20150720_1200', 'data.20150721_1200', 'data.20150722_1200', 'data.20150723_1200', 'data.20150724_1200', 'data.20150727_1200', 'data.20150728_1200', 'data.20150729_1200', 'data.20150730_1200', 'data.20150731_1200']



In [32]:
importlib.reload(test_helper)
ols_model = test_helper.Regression(data_path = FILE_PATH)
ols_model.train(train_df, feature_col_names = FEATURE_COL_NAMES, interacting_terms_list = list_of_interacting_terms, hyperparam_dict={'fit_intercept': False, 'positive':True})
ols_model.get_metric(dataframes = test_dfs)

No YYYYMMDD datetime matched.

You're using: OLS.

Available hyperparams: {'fit_intercept': True, 'copy_X': True, 'n_jobs': None, 'positive': False}
Features being used: ["('relvol_nt_0', 'rrirpnxm_nt_0')", "('relvol_lst15_0', 'rrirpnxm_lst15_0')", "('relvol_lsthrx15_0', 'rrirpnxm_lsthrx15_0')", "('relvol_toxhr_0', 'rrirpnxm_toxhr_0')", 'rrirpnxm_nt_0', 'rrirpnxm_lst15_0', 'rrirpnxm_lsthrx15_0', 'rrirpnxm_toxhr_0']


(0.03106576056947642, 0.00023269608826127447, 0.9954860021549081)

In [33]:
importlib.reload(test_helper)
full_ols_model = test_helper.Regression(data_path = FILE_PATH)
full_ols_model.train(train_df)
full_ols_model.get_metric(test_dfs)

No YYYYMMDD datetime matched.

You're using: OLS.

Available hyperparams: {'fit_intercept': True, 'copy_X': True, 'n_jobs': None, 'positive': False}
Features being used: ['eqid', 'wt', 'today', 'tmwam', 'dn3sttmwmd', 'dn1', 'dn4x1', 'cftorrrelstd_open_0', 'liqlog_open_0', 'llirpnxm_am_1', 'llirpnxm_dy_12to16', 'llirpnxm_dy_17to21', 'llirpnxm_dy_1to3', 'llirpnxm_dy_4to6', 'llirpnxm_dy_7to11', 'llirpnxm_lst15_0', 'llirpnxm_lsthrx15_0', 'llirpnxm_md_1', 'llirpnxm_nt_0', 'llirpnxm_nt_1', 'llirpnxm_nt_12to16', 'llirpnxm_nt_17to21', 'llirpnxm_nt_1to3', 'llirpnxm_nt_4to6', 'llirpnxm_nt_7to11', 'llirpnxm_pm_1', 'llirpnxm_toxhr_0', 'mocrelvol_open_1', 'mocrelvol_open_13to17', 'mocrelvol_open_18to22', 'mocrelvol_open_2to4', 'mocrelvol_open_5to7', 'mocrelvol_open_8to12', 'momr10d_open_0', 'momr50dx10d_open_0', 'moorelvol_open_0', 'moorelvol_open_12to16', 'moorelvol_open_17to21', 'moorelvol_open_1to3', 'moorelvol_open_4to6', 'moorelvol_open_7to11', 'nnetticksrelmultstdev_am_1', 'nnetticksrelmultst

(0.9209163827296906, 0.0067170703912272835, 1.016981924374905)

In [34]:
importlib.reload(test_helper)
lasso_model = test_helper.Regression(regression_type="Lasso", data_path=FILE_PATH )
lasso_model.train(train_df, feature_col_names = FEATURE_COL_NAMES, interacting_terms_list = list_of_interacting_terms, hyperparam_dict={'random_state': 1})
lasso_model.get_metric(dataframes = test_dfs)

No YYYYMMDD datetime matched.

You're using: LASSO.

Available hyperparams: {'eps': 0.001, 'n_alphas': 100, 'alphas': None, 'fit_intercept': True, 'precompute': 'auto', 'max_iter': 1000, 'tol': 0.0001, 'copy_X': True, 'cv': None, 'verbose': False, 'n_jobs': None, 'positive': False, 'random_state': None, 'selection': 'cyclic'}
Features being used: ["('relvol_nt_0', 'rrirpnxm_nt_0')", "('relvol_lst15_0', 'rrirpnxm_lst15_0')", "('relvol_lsthrx15_0', 'rrirpnxm_lsthrx15_0')", "('relvol_toxhr_0', 'rrirpnxm_toxhr_0')", 'rrirpnxm_nt_0', 'rrirpnxm_lst15_0', 'rrirpnxm_lsthrx15_0', 'rrirpnxm_toxhr_0']


(0.032073395564528505, 0.00024760358885082384, 1.1173959088759213)

In [37]:
# Lasso Experiment:
importlib.reload(test_helper)
# No hyperparams in this exp
test_helper.lasso_exp(FILE_PATH, train_df = train_df, test_dfs = test_dfs, interacting_terms_list = list_of_interacting_terms, feature_col_names = FEATURE_COL_NAMES, num_trials = 15)

Conducting 15 on a Lasso model with hyperparams:
None
No YYYYMMDD datetime matched.

You're using: LASSO.

Available hyperparams: {'eps': 0.001, 'n_alphas': 100, 'alphas': None, 'fit_intercept': True, 'precompute': 'auto', 'max_iter': 1000, 'tol': 0.0001, 'copy_X': True, 'cv': None, 'verbose': False, 'n_jobs': None, 'positive': False, 'random_state': None, 'selection': 'cyclic'}
Features being used: ["('relvol_nt_0', 'rrirpnxm_nt_0')", "('relvol_lst15_0', 'rrirpnxm_lst15_0')", "('relvol_lsthrx15_0', 'rrirpnxm_lsthrx15_0')", "('relvol_toxhr_0', 'rrirpnxm_toxhr_0')", 'rrirpnxm_nt_0', 'rrirpnxm_lst15_0', 'rrirpnxm_lsthrx15_0', 'rrirpnxm_toxhr_0']
No YYYYMMDD datetime matched.

You're using: LASSO.

Available hyperparams: {'eps': 0.001, 'n_alphas': 100, 'alphas': None, 'fit_intercept': True, 'precompute': 'auto', 'max_iter': 1000, 'tol': 0.0001, 'copy_X': True, 'cv': None, 'verbose': False, 'n_jobs': None, 'positive': False, 'random_state': None, 'selection': 'cyclic'}
Features being used:

In [40]:
importlib.reload(test_helper)
xgb_model = test_helper.Regression(regression_type="XGBoost", data_path=FILE_PATH)
xgb_model.train(train_df, feature_col_names = FEATURE_COL_NAMES, interacting_terms_list = list_of_interacting_terms, hyperparam_dict={'random_state': 1})
xgb_model.get_metric(test_dfs)

No YYYYMMDD datetime matched.

You're using: XGBOOST.

Available hyperparams: {'n_estimators': None, 'objective': 'reg:squarederror', 'max_depth': None, 'max_leaves': None, 'max_bin': None, 'grow_policy': None, 'learning_rate': None, 'verbosity': None, 'booster': None, 'tree_method': None, 'gamma': None, 'min_child_weight': None, 'max_delta_step': None, 'subsample': None, 'sampling_method': None, 'colsample_bytree': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'reg_alpha': None, 'reg_lambda': None, 'scale_pos_weight': None, 'base_score': None, 'missing': nan, 'num_parallel_tree': None, 'random_state': None, 'n_jobs': None, 'monotone_constraints': None, 'interaction_constraints': None, 'importance_type': None, 'device': None, 'validate_parameters': None, 'enable_categorical': False, 'feature_types': None, 'max_cat_to_onehot': None, 'max_cat_threshold': None, 'multi_strategy': None, 'eval_metric': None, 'early_stopping_rounds': None, 'callbacks': None}
Features being used: 

(0.011482916871108395, 0.00018141080745390475, 0.12422154)